### 3 problems we have
1. We have 200 shuffle partitions but only few of them have data
2. when there is a dominating key, and there is a partition skew
3. I want to join orders with of distinct customers

### Solutiins to above 3 problems

#AQE provides
============
1. dynamically coalesing the number of shuffle partitions
2. dynamically handling partition skew
3. dynamically switching join strategies

### number of records:

1. size of the data
2. min and max of each column
3. count of number of occurences of each key
4. if data is reduced during the join it dynamically switch the join strategies like broadcast join

In [1]:
# Problem no:1 We have 200 shuffle partitions but only few of them have data
#solution: dynamically coalesing the number of shuffle partitions

In [30]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/itv015970/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [31]:
order_schema = "order_id long, order_date string, customer_id long, order_status string" 

In [32]:
orders_df = spark.read \
.format("csv") \
.schema(order_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [34]:
orders_df.groupBy("order_status").count().write.format("noop").mode("overwrite").save()

In [35]:
# check adaptive query is enabled or not

In [36]:
spark.conf.get("spark.sql.adaptive.enabled")

'false'

In [37]:
spark.conf.set("spark.sql.adaptive.enabled", True)

In [38]:
orders_df.groupBy("order_status").count().write.format("noop").mode("overwrite").save()